## Imports

In [6]:

import numpy as np
import wfdb
from scipy.signal import butter, filtfilt, find_peaks
import os

## Desarrollo

### Funciones necesarias

In [ ]:
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

### Lectura de 1 registro

In [ ]:
record = wfdb.rdrecord('e0103', directory='/datos/EDB')

fs = record.fs
signal = record.p_signal[:,0]

# Procesar ECG
filtered_ecg = bandpass_filter(signal, 0.5, 40, fs)

### Detección picos R

In [ ]:
distance = int(0.4 * fs)
peaks, _ = find_peaks(filtered_ecg, distance=distance, height=filtered_ecg.std())

### Intervalos RR

In [ ]:
rr_intervals = np.diff(peaks) / fs # segundos

# Para guardar los RR de todos los archivos

all_rr = []
base_path = '/datos/EDB'

for n in range(1, 91):
    rec_id = f'e{100 + n:04d}'  # e0101 hasta e0190
    rec_id = f'e{n:04d}'

    hea_path = os.path.join(base_path, rec_id + '.hea')
    dat_path = os.path.join(base_path, rec_id + '.dat')

    if not (os.path.exists(hea_path) and os.path.exists(dat_path)):
        print(f'Saltando {rec_id} (no encontrado)')
        continue

    try:
        record = wfdb.rdrecord(rec_id, directory=base_path)
        signal = record.p_signal[:, 0]
        fs = record.fs

        ecg = bandpass_filter(signal, 0.5, 40, fs)
        peaks, _ = find_peaks(ecg, distance=int(0.4 * fs), height=np.std(ecg))
        rr = np.diff(peaks) / fs

        if len(rr) > 50:
            all_rr.append(rr)
        else:
            all_rr.append(None)
    except Exception as e:
        print(f'Error con {rec_id}: {e}')
        all_rr.append(None)


### Aplicar estimador de Hurst

In [ ]:
# Código aquí :v

# Actividad 1

## Prueba de sesgo

## Prueba de robustez

## Prueba de sesgo